# Exercise 1
Given:
- $C = 20$  
- $e = 13$
- $n = 77$

$n$ can be a produt of two primes 7 and 11. So:
$$
n = p * q = 7 * 11
$$
Our next step is to calculate the $\varphi(n)$: 
$$
\varphi(77) = (7 - 1) * (11 - 1) = 60  
$$
$$
\varphi(77) = 60
$$

To calculate the private key we need to solve this equation:
$$
e * d = 1 \mod(\varphi(n))
$$
$$
13 * d = 1 \mod(\varphi(77))
$$
$$
13 * d = 1 \mod(60)
$$
$$
13 * d = 1 + k * \varphi(77)
$$
$$
1 = 13*d - k * 60
$$

To solve this problem, we can use the Extended Euclidian Algorithm:
$$
60 = 13 * 4 + 8
$$
$$
13 = 8 * 1 + 5
$$
$$
8 = 5 * 1 + 3
$$
$$
5 = 3 * 1 + 2
$$
$$
3 = 2 * 1 + 1
$$
And now we do the reverse:
$$
1 = 3 - 2 * 1
$$
$$
1 = 3 - (5 - 3 * 1) * 1
$$
$$
1 = 2 * 3 - 5 
$$
$$
1 = 2 * (8 - 5 * 1) - 5
$$
$$
1 = 2*8 - 2*5 - 5
$$
$$
1 = 2*8 - 3*5
$$
$$
1 = 2*8 - 3*(13 - 8*1)
$$
$$
1 = 2*8 - 3*13 + 3*8
$$
$$
1 = 5*8 - 3*13
$$
$$
1 = 5*(60 - 4*13) - 3*13
$$
$$
1 = 5*60 - 20*13 - 3*13
$$
$$
1 = 5*60 - 23*13
$$
Given the previous formula $1 = 13*d - k * 60$, we can say that:
- $d \equiv -23 \equiv  37 \mod 60$ (private key)
- k = -5

To decrypt, we need to solve this problem:
$$
M = C^{d} \mod n
$$
$$
M = 20^{37} \mod 77
$$
$$
M = 48
$$

# Exercise 2
To determine the private key 'd' from the public key (e=65, n=2881), the first step is to factor the modulus 'n' to find its prime components, 'p' and 'q'.

The factorization of n=2881 results in p=43 and q=67.

With these values, φ(n) is calculated:
$$
φ(n) = (p-1) * (q-1) = (43-1) * (67-1) = 42 * 66 = 2772
$$
The private key 'd' is the multiplicative inverse of 'e mod φ(n)'. Thus, it comes down to solving:
$$
65 * d ≡ 1 \pmod{2772}
$$
Solving this equation results in d = 725.

Therefore, the user's private key is 725.

# Exercise 3
No. Encrypting a small, predictable number like a phone number with textbook RSA is not secure. The message space is tiny, so an attacker can easily try all possible phone numbers (a dictionary attack) using the public key until the ciphertext matches. Additionally, with common small public exponents (like $𝑒 = 3$), if the number is small enough that $𝑚^{𝑒} < 𝑛$, the attacker can simply take the integer $e-th$ root of the ciphertext to recover the message.

# Exercise 4: The Miller-Rabin Primality Test

Exercise 4 asks for a Python program based on the strategy described in the provided text. This strategy is the **Miller-Rabin primality test**, which is a probabilistic test utilized to efficiently determine if a large number $n$ is prime.

The algorithm relies on the logic established in Theorems 1 and 2 of the text. The process is as follows:

1.  **Decomposition**:
    First, we take the number $n-1$ (where $n$ is the number we want to test) and write it in the form $n-1 = 2^k d$, where $d$ is an odd number.

2.  **Witness Selection**:
    We pick a random integer $a$ (the "witness") such that $1 < a < n-1$.

3.  **Test Conditions**:
    According to Theorem 2, if $n$ is prime, then one of the following conditions must be true:
    * $a^d \equiv 1 \pmod{n}$
    * $a^{2^i d} \equiv -1 \pmod{n}$ for some $i$ in the range $0 \le i \le k-1$.

4.  **Conclusion**:
    * If $a$ satisfies **either** of these conditions, it is not a witness to $n$'s compositeness, and $n$ might be prime.
    * If $a$ satisfies **neither** condition, it *is* a witness. We have definitively proven that $n$ is composite.

5.  **Iteration**:
    We repeat this test $m$ times with $m$ different random witnesses. If $n$ passes all $m$ tests, we can be confident it is prime with a probability of $1 - 2^{-m}$.

**miller-rabbin.py**:

In [1]:
import random

def check_witness(a, n, k, d):
    """
    This function checks if 'a' is a witness for the compositeness of 'n'.

    Returns True if n PASSES the test (is probably prime).
    Returns False if n FAILS the test (is composite).
    """
    
    # calculate x = a^d (mod n)
    x = pow(a, d, n)
    
    # condition (a): a^d ≡ 1 (mod n) 
    # also check a^d ≡ -1 (mod n), which is the i=0 case for condition (b)
    if x == 1 or x == n - 1:
        return True

    # check condition (b) for i = 1 to k-1
    # loop k-1 times, squaring x each time
    for _ in range(1, k):
        # x = x^2 (mod n)
        # calculates a^(2d), a^(4d), ..., a^(2^(k-1)d)
        x = pow(x, 2, n)

        # if x ≡ -1 (mod n), condition (b) is met
        if x == n - 1:
            return True

        # If x ≡ 1 (mod n), but the previous value wasn't -1,
        # found a non-trivial square root of 1, so n is composite.
        if x == 1:
            return False 
            
    return False  # n is definitely composite

def miller_rabin(n, m=40):
    """
    Miller-Rabin primality test.
    'n' is the number to test.
    'm' is the number of iterations.
    """
    
    # handle trivial cases
    if n <= 1:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False  # n is an even number > 2
        
    # 1. decompose n-1 into 2^k * d
    d = n - 1
    k = 0
    while d % 2 == 0:
        d //= 2
        k += 1

    # 2. iterate m times
    for _ in range(m):
        # 3. pick a random witness 'a'
        a = random.randint(2, n - 2)
        
        # 4. check if 'a' is a witness
        if not check_witness(a, n, k, d):
            return False  # n is definitely composite
            
    # 5. if all m tests pass, n is probably prime
    return True


# test a known large prime
prime_1 = 104395301
print(f"Is {prime_1} prime? {miller_rabin(prime_1)}")

# test another known large prime
prime_2 = 2147483647 
print(f"Is {prime_2} prime? {miller_rabin(prime_2)}")

# test a large composite number (product of two primes)
composite = prime_1 * prime_2
print(f"Is {composite} prime? {miller_rabin(composite)}")

# test a small composite
print(f"Is 561 prime? {miller_rabin(561)}")

# test trivial cases
print(f"Is 2 prime? {miller_rabin(2)}")
print(f"Is 3 prime? {miller_rabin(3)}")
print(f"Is 100 prime? {miller_rabin(100)}")

Is 104395301 prime? True
Is 2147483647 prime? True
Is 224187201721142747 prime? False
Is 561 prime? False
Is 2 prime? True
Is 3 prime? True
Is 100 prime? False


# Exercise 5
No. Reusing the same modulus 'n' after the private key has been leaked is insecure.

The misconception is to think that only 'd' is secret. In reality, the security of RSA depends on the factors of 'n' (the primes 'p' and 'q').

When the pair (e, d) is compromised, an attacker can use this information to efficiently factor 'n'. From 'p' and 'q', they can calculate φ(n) and can derive any new private key, even if Bob chooses a new public exponent (e').

This means that all key pairs generated with the same modulus are automatically compromised.

Thus, if the private key is compromised, the modulus 'n' is no longer secure. It is mandatory to generate new primes and a new modulus, not just new exponents.
